In [ ]:
# Import required libraries for data processing and pattern analysis
import pandas as pd  # Data manipulation and analysis
import re            # Regular expressions for text processing

In [ ]:
# Load transaction data from Excel file
# Update the file path to match your transaction data location
df = pd.read_excel(
    "C:\\Users\\subha\\Downloads\\MERGED_6 Banks_20250901_to_20250902_Cateogry Flagging MF.xlsx"
)

In [ ]:
# Sort data by Group for better organization and analysis
df = (
    df
    .sort_values(['Group'])
)

In [ ]:
# Convert Date column to datetime format for time-based analysis
# Handles missing values with coerce parameter
df['Date'] = pd.to_datetime(
    df['Date'],
    errors='coerce',
    format='%Y-%m-%d'   # works for yy-mm-dd too
)

In [ ]:
# Extract temporal features from dates for pattern analysis
df['day'] = df['Date'].dt.day                                    # Day of month (1-31)
df['week'] = df['Date'].dt.isocalendar().week.astype(int)       # ISO week number

# Create Period objects for grouping (needed before conversion to string)
df['month'] = df['Date'].dt.to_period('M')      # Year-Month period
df['quarter'] = df['Date'].dt.to_period('Q')    # Year-Quarter period

In [ ]:
# Convert Period objects to strings for analysis and export
df['month'] = df['month'].astype(str)
df['quarter'] = df['quarter'].astype(str)

In [ ]:
# Detect MONTHLY patterns - transactions with consistent day-of-month
# Criteria: Occur in at least 3 different months AND day variation ≤ 3 days
monthly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          months=('month', 'nunique'),              # Count unique months
          day_std=('day', 'std')                     # Standard deviation of day
      )
)

# Filter for transactions meeting monthly criteria
monthly_candidates = monthly_pattern[
    (monthly_pattern['months'] >= 3) &             # At least 3 months
    (monthly_pattern['day_std'] <= 3)              # Consistent day (within 3 days)
].assign(periodicity='Monthly')

In [ ]:
# Detect WEEKLY patterns - transactions on the same day of the week
# Criteria: Occur in at least 4 different weeks AND on the same weekday (no variation)
df['weekday'] = df['Date'].dt.weekday    # 0=Monday, 6=Sunday

weekly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          weeks=('week', 'nunique'),                # Count unique weeks
          weekday_std=('weekday', 'std')            # Standard deviation of weekday
      )
)

# Filter for transactions meeting weekly criteria
weekly_candidates = weekly_pattern[
    (weekly_pattern['weeks'] >= 4) &               # At least 4 weeks
    (weekly_pattern['weekday_std'] == 0)           # Same weekday always
].assign(periodicity='Weekly')

In [ ]:
# Detect QUARTERLY patterns - transactions occurring at quarterly intervals
# Criteria: Occur in at least 2 different quarters
quarterly_pattern = (
    df.groupby('Cleaned Narration')
      .agg(
          quarters=('quarter', 'nunique')           # Count unique quarters
      )
)

# Filter for transactions meeting quarterly criteria
quarterly_candidates = quarterly_pattern[
    quarterly_pattern['quarters'] >= 2             # At least 2 quarters
].assign(periodicity='Quarterly')

In [ ]:
# Combine all periodicity patterns with priority: Monthly > Weekly > Quarterly
# This creates a mapping of transaction descriptions to their detected periodicity
periodicity_map = (
    pd.concat([
        monthly_candidates[['periodicity']],       # Monthly patterns first (highest priority)
        weekly_candidates[['periodicity']],        # Weekly patterns second
        quarterly_candidates[['periodicity']]      # Quarterly patterns last
    ])
    .reset_index()                                  # Bring group key (Cleaned Narration) back
    .drop_duplicates(
        subset='Cleaned Narration',                # Remove duplicates by transaction description
        keep='first'                               # Keep first occurrence (respects priority order)
    )
    .set_index('Cleaned Narration')               # Create lookup map
)

In [ ]:
# Apply periodicity classification to each transaction
# Transactions not matching any pattern are labeled as 'Irregular'
df['periodicity'] = df['Cleaned Narration'].map(
    periodicity_map['periodicity']
).fillna('Irregular')

In [ ]:
# Convert Date back to string format (YYYY-MM-DD) for export
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
# Prepare data for export: remove temporary analysis columns
df_export = df.copy()
cols_to_drop = ['day', 'week', 'month', 'quarter', 'weekday']  # Remove intermediate features
df_export = df_export.drop(columns=cols_to_drop, errors='ignore')

In [ ]:
# Export results to Excel file with periodicity classifications
df_export.to_excel("final_output.xlsx", index=False)